Steps:
- Scarping Job postings with Selenium and bs4
- Visualize in Tableau or PowerBI

import necessary modules - first try with BeautifulSoup

In [71]:
import driver as driver
import pandas as pd
import re
import lxml
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
import datetime
start_time = time()

from warnings import warn


Select wanted url

In [72]:
job_kind = str(input('Please enter the kind of job you want to work in (e.g. Data Analyst)'))
city = str(input('Please enter the city you want to work in'))
state = str(input('Please enter the state you want to work in'))
country = str(input('Please enter the country you want to work in'))
url = 'https://www.linkedin.com/jobs/search?keywords=' + job_kind + '&location='+ city + '%2C%20'+ state +'%2C%20'+ country

no_of_jobs = 51

url



'https://www.linkedin.com/jobs/search?keywords=Consulting&location=Berlin%2C%20%2C%20Germany'

open windows with url stated above in

In [73]:
driver = webdriver.Chrome()
driver.get(url)
sleep(3)
action = ActionChains(driver)

to show more jobs than the initial 25

In [74]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
sleep(20)



parsing the visible webpage

In [75]:
pageSource = driver.page_source
soup = BeautifulSoup(pageSource,  "html.parser")



Parsing content

In [76]:
first_job = soup.find('h3', class_ = 'result-card__title job-result-card__title').text
print(first_job)

first_company = soup.find('a', class_ = 'result-card__subtitle-link job-result-card__subtitle-link').text
print(first_company)

Strategy Consultant
Zalando SE


Create job container

In [77]:
job_container = soup.find('ul', class_="jobs-search__results-list")
print(len(job_container))

46


create for loop to look for all jobs

In [78]:
print('You are scraping information about {} jobs.'.format(len(job_container)))

You are scraping information about 46 jobs.


In [79]:
job_titles = []
companies = []
locations = []
time = []
date = []


for jobs in job_container:
    post_title = jobs.find('h3', class_ = 'result-card__title job-result-card__title').text
    job_titles.append(post_title)

    post_company = jobs.find('h4', class_ = 'result-card__subtitle job-result-card__subtitle').text
    companies.append(post_company)

    post_locations = jobs.find('span', class_ = 'job-result-card__location').text
    locations.append(post_locations)

    post_date = jobs.select_one('time')['datetime']
    date.append(post_date)


In [80]:
ml = {'Title': job_titles, 'Company': companies, 'Locations': locations, 'Post Date': date}
print(ml)

{'Title': ['Strategy Consultant', 'Associate', 'Junior Consultant - Risk Management', 'Associate Consultant (m / f / d)', 'Operations Manager', 'Economic Consulting Research Officer (m/f/d) Energy, Environment, Communications and Infrastructure', 'Head of Consulting (m/f/x) ', 'Project Manager Innovation - Culture Transformation/Learning (m/f/x)', 'Investment Analyst', 'Energy Policy Internship – DACH (m/w/d)', 'Graduate Professional Services Consultant', 'Business Operations and Strategy Associate (f/m/d)', 'Chief of Staff', 'Jr. Analyst', 'Investor', 'Organisational Development (various levels)', 'Internship', 'Senior Associate', 'Senior Associate Consultant (m/f/x) | Berlin', 'Strategic Project Lead (m/f/d)', 'Business Analyst / Consultant (m/w/d) Public', '(Associate) Consultant (gn)', 'Digital Transformation Consultant, Google Cloud', 'BERLIN - Chief of Staff (m/f/x)', 'Business Program Manager', 'Investment Specialist', 'Strategic Project Lead (m/f/d)', 'Consultant - Risk Managem

In [81]:
df = pd.DataFrame(ml)
df.head(1)

,Title,Company,Locations,Post Date
0,Strategy Consultant,Zalando SE,"Berlin, Berlin, Germany",2021-01-28


Separate City from State and Country


In [82]:
new = df.Locations.str.split(',', n = 0, expand = True)

In [83]:
new2 = new.rename(columns = {0: 'City2'})

In [84]:
new2


,City2,1,2
0,Berlin,Berlin,Germany
1,Berlin,Germany,None
2,Berlin,Berlin,Germany
3,Berlin,Berlin,Germany
4,Berlin,Germany,None
5,Berlin,Germany,None
6,Berlin,Germany,None
7,Berlin,Germany,None
8,Berlin,Berlin,Germany
9,Berlin,Berlin,Germany


merge df and new, drop Locations as it is redundant now

In [85]:
df2 = df.join(new2, how = 'left')
df2.drop(columns = ['Locations',1 , 2], inplace = True)
df2.head(4)

,Title,Company,Post Date,City2
0,Strategy Consultant,Zalando SE,2021-01-28,Berlin
1,Associate,Atheneum,2021-01-26,Berlin
2,Junior Consultant - Risk Management,McKinsey & Company,2020-11-26,Berlin
3,Associate Consultant (m / f / d),UpSlide,2021-01-29,Berlin


In [86]:
new3 = df2.City2.str.split(' ', expand=True)
new4 = new3.rename(columns = {0: 'City'})
new5 = new4.City
new5.head()

0    Berlin
1    Berlin
2    Berlin
3    Berlin
4    Berlin
Name: City, dtype: object

In [87]:
df4 = df2.join(new5, how = 'left')
df4.drop(columns = ["City2"], inplace = True)
df4.set_index('Title', inplace = True)
df4

,Company,Post Date,City
Title,,,
Strategy Consultant,Zalando SE,2021-01-28,Berlin
Associate,Atheneum,2021-01-26,Berlin
Junior Consultant - Risk Management,McKinsey & Company,2020-11-26,Berlin
Associate Consultant (m / f / d),UpSlide,2021-01-29,Berlin
Operations Manager,Approach People Recruitment,2021-02-03,Berlin
"Economic Consulting Research Officer (m/f/d) Energy, Environment, Communications and Infrastructure",NERA Economic Consulting,2021-02-03,Berlin
Head of Consulting (m/f/x),Hubject,2021-01-24,Berlin
Project Manager Innovation - Culture Transformation/Learning (m/f/x),Forever Day One,2021-01-31,Berlin
Investment Analyst,AVOMIND,2020-12-17,Berlin


create CSV, with todays Date

In [88]:
date_object = datetime.date.today()
date = str(date_object) + '_'
date

df4.to_csv('C:/Users/Marc/Dropbox/01_Marc Arnold/Python Projects/04_Job Viz/02_Output/' + date + city + '_' + country + '.csv')

concat new jobs to existing list

In [89]:
final_df = pd.read_csv('C:/Users/Marc/Dropbox/01_Marc Arnold/Python Projects/04_Job Viz/02_Output/01_Final DF/Final_DF_LinkedIN Jobs.csv')
final_df.set_index('Title', inplace = True)
final_df.head()

,Company,Post Date,City
Title,,,
Data Analyst,EPAM Systems,2021-01-15,Zurich
Global Digital Data Analyst,La Prairie Group AG,2021-01-17,Zurich
Business Analyst,Tata Consultancy Services,2021-01-14,Zurich
Business Intelligence Analyst (80%),Spitex Zürich Limmat,2021-01-22,Zurich
"Business Analyst (m/w), Zurich",MoneyPark,2021-01-22,Schlatt


In [90]:
dfWIP = pd.concat([final_df, df4], axis=0)
dfWIP.head()

,Company,Post Date,City
Title,,,
Data Analyst,EPAM Systems,2021-01-15,Zurich
Global Digital Data Analyst,La Prairie Group AG,2021-01-17,Zurich
Business Analyst,Tata Consultancy Services,2021-01-14,Zurich
Business Intelligence Analyst (80%),Spitex Zürich Limmat,2021-01-22,Zurich
"Business Analyst (m/w), Zurich",MoneyPark,2021-01-22,Schlatt


drop duplicated rows

In [91]:
new_final_df = dfWIP.drop_duplicates()
new_final_df

,Company,Post Date,City
Title,,,
Data Analyst,EPAM Systems,2021-01-15,Zurich
Global Digital Data Analyst,La Prairie Group AG,2021-01-17,Zurich
Business Analyst,Tata Consultancy Services,2021-01-14,Zurich
Business Intelligence Analyst (80%),Spitex Zürich Limmat,2021-01-22,Zurich
"Business Analyst (m/w), Zurich",MoneyPark,2021-01-22,Schlatt
...,...,...,...
Intern Management Consulting (m/f/d),scaleon,2021-02-03,Berlin
Strategy & Finance Associate,scoutbee,2021-01-12,Berlin
Co-Founder / Managing Director,Excelling Ventures,2021-01-22,Berlin


In [92]:
new_final_df.to_csv('C:/Users/Marc/Dropbox/01_Marc Arnold/Python Projects/04_Job Viz/02_Output/01_Final DF/Final_DF_LinkedIN Jobs.csv')

